Почти все клавиши в каждой сетке (grid) имеют ширину на 1 больше необходимой

Этот блокнот демонстрирует данную проблему 

Возьмем клавиши одного ряда и упорядочим их по возрастанию x. Если клавиши составаляют плотную сетку без зазоров (как оно обычно и бывает), то `key[i].x + key[i].w == key[i+1].x`. Однако, в датасете для абсолютно всех клавиш (кроме последней клавиши каждого ряда) `key[i].x + key[i].w = key[i+1].x + 1`. Я предполагаю, что это ошибка, и что вместо `key[i].w` должно быть `key[i].w - 1`. Чтобы исправить все такие ситуации предлагаю:
1. Разделить клавиши на ряды (по y).
2. Упорядочить клавиши в каждом ряду по x
4. Для всех `i` если `(key[i].x + key[i].w) - key[i+1].x == 1`,  `key[i].w -= 1`.
5. Для последней клавиши чиним ширину так: `right_padding = grid['weight'] - key[-1].x - key[-1].w`. `right_padding = key[0].x`. Если `right_padding - left_padding == 1`, то `key[-1].w -= 1`. 
6. объединить все ряды обратно в один список клавиш.
7. Сохранить результат в новый файл.
8. Сравнить optimized_nkl с nkl_old_gridfile и nkl_new_gridfile.


In [ ]:
from typing import List, Dict
import json

def group_keys_by_y(keys: List[dict]) -> Dict[int, List[dict]]:
    y_to_keys = {}
    for key in keys:
        y = key['hitbox']['y']
        y_to_keys.setdefault(y, []).append(key)
    return y_to_keys


GRID_NAME_TO_GRID_PATH = '../../data/data_separated_grid/gridname_to_grid.json'

with open(GRID_NAME_TO_GRID_PATH, encoding='utf-8') as f:
    gridname_to_grid = json.load(f)

for g_name, grid in gridname_to_grid.items():
        
    grid = gridname_to_grid['default']

    keys = grid['keys']

    y_to_keys = group_keys_by_y(keys)
    key_dict_rows = list(y_to_keys.values())

    for row in key_dict_rows:
        row.sort(key=lambda key: key['hitbox']['x'])



    value_counts = {}

    for keys in key_dict_rows:
        for i in range(len(keys) - 1):
            key1 = keys[i]
            key2 = keys[i + 1]
            x1 = key1['hitbox']['x']
            w1 = key1['hitbox']['w']
            x2 = key2['hitbox']['x']
            diff = x1 + w1 - x2
            value_counts.setdefault(diff, 0)
            value_counts[diff] += 1

    for keys in key_dict_rows:
        padding_left = keys[0]['hitbox']['x']
        padding_right = grid['width'] - (keys[-1]['hitbox']['x'] + keys[-1]['hitbox']['w'])
        diff = padding_right - padding_left
        value_counts.setdefault(diff, 0)
        value_counts[diff] += 1

    
    print(g_name)
    print(value_counts)

# Out:
# extra
# {1: 35, 0: 4}
# default
# {1: 35, 0: 4}